In [ ]:
import pandas as pd
from pprint import pprint
from comprehend_detect import ComprehendDetect
from detect_pi_si import *
from PIDetectAnalysis import *
import boto3
from botocore.exceptions import ClientError
import warnings
warnings.filterwarnings('ignore')
comprehend_client = boto3.client("comprehend")

In [11]:
text = "Sean Dyer at 4972 w. 129th Ter. Leawood, KS has ssn 489864926 credit card number 4734391837502655 analyticmodels@gmail.com https://analyticmodels.net"
comp_detect = ComprehendDetect(comprehend_client)
pii_list = comp_detect.detect_pii(text, 'en')
pprint(pii_list)

[{'BeginOffset': 0,
  'EndOffset': 9,
  'Score': 0.9999914169311523,
  'Type': 'NAME'},
 {'BeginOffset': 13,
  'EndOffset': 43,
  'Score': 0.9999077916145325,
  'Type': 'ADDRESS'},
 {'BeginOffset': 52,
  'EndOffset': 61,
  'Score': 0.9999984502792358,
  'Type': 'SSN'},
 {'BeginOffset': 81,
  'EndOffset': 97,
  'Score': 0.9999816417694092,
  'Type': 'CREDIT_DEBIT_NUMBER'},
 {'BeginOffset': 98,
  'EndOffset': 122,
  'Score': 0.999990701675415,
  'Type': 'EMAIL'},
 {'BeginOffset': 123,
  'EndOffset': 149,
  'Score': 0.9999992251396179,
  'Type': 'URL'}]


### Run Comprehend detection and redaction on 1000 rows of sample data in S3

In [ ]:
df = pd.read_csv("s3://pearsoncomprehend/comprehendData/pii_test_faker.csv",delimiter=",")
df.dropna()

#instantiate wrapper class
comp_detect = ComprehendDetect(comprehend_client)

#return PII detections
df_pi = detect_pi(df,comp_detect)

#redact PII detections
df_redacted = redact_df(df,df_pi)

#rescan for excluded names
df_names = detect_names(df_redacted,comp_detect)

#redact names
df_redacted = redact_df(df_redacted,df_names)


generator = PIDetectionReportGenerator("s3://pearsoncomprehend/comprehendData/detectedPI.csv")
generator.generate_markdown_report()

Markdown report generated successfully: reports/PI_detection_report.md


'reports/PI_detection_report.md'

### Write result tables to S3

In [ ]:
df_pi.to_csv("s3://pearsoncomprehend/comprehendData/detectedPI.csv",index=False)
df_names.to_csv("s3://pearsoncomprehend/comprehendData/detectedNames.csv",index=False)
df_redacted.to_csv("s3://pearsoncomprehend/comprehendData/fakerRedacted.csv",index=False)